In [1]:
import sys
import tkinter
import pandas as pd
import numpy as np
import tkinter.messagebox
from tkintermapview import TkinterMapView
from pyswip import Prolog

In [2]:
df = pd.read_csv('./Destinations.csv')
print(df.shape)
df.head()

(102, 13)


,Destinations,country,region,Climate,Budget,Activity,Demographics,Duration,Cuisine,History,Natural Wonder,Accommodation,Language
0,Tokyo,Japan,East Asia,Temperate,High,Cultural,Solo,Long,Asian,Modern,Mountains,Luxury,Japanese
1,Ottawa,Canada,North America,Cold,Medium,Adventure,Family-friendly,Medium,European,Modern,Forests,Mid-range,English
2,Mexico City,Mexico,North America,Temperate,Low,Cultural,Senior,Short,Latin American,Ancient,Mountains,Budget,Spanish
3,Rome,Italy,Southern Europe,Temperate,High,Cultural,Solo,Medium,European,Ancient,Beaches,Luxury,Italian
4,Brasilia,Brazil,South America,Tropical,Low,Adventure,Family-friendly,Long,Latin American,Modern,Beaches,Budget,Portuguese


In [3]:
adj = pd.read_csv('./Adjacency_matrix.csv')
print(adj.shape)
adj.head()

(102, 103)


,Destinations,Tokyo,Ottawa,Mexico City,Rome,Brasilia,Canberra,New Delhi,Pretoria,Madrid,...,Bursa,Munich,Hamburg,Frankfurt,Alexandria,Luxor,Aswan,Christchurch,Queenstown,Rotorua
0,Tokyo,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
1,Ottawa,1,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2,Mexico City,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
3,Rome,1,0,0,0,1,1,1,0,0,...,1,0,1,0,0,0,0,0,1,0
4,Brasilia,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [4]:
# TODO 1: read destinations' descriptions from Destinations.csv and add them to the prolog knowledge base
################################################################################################
# STEP1: Define the knowledge base of illnesses and their symptoms

def clear_KB(prolog):
    prolog.retractall("destinations('_,_')")
    prolog.retractall("country(_,_)")
    prolog.retractall("region(_,_)")
    prolog.retractall("climate(_,_)")
    prolog.retractall("budget(_,_)")
    prolog.retractall("activity(_,_)")
    prolog.retractall("demographics(_,_)")
    prolog.retractall("duration(_,_)")
    prolog.retractall("cuisine(_,_)")
    prolog.retractall("history(_,_)")
    prolog.retractall("natural_wonder(_,_)")
    prolog.retractall("accommodation(_,_)")
    prolog.retractall("language(_,_)")


prolog = Prolog()
clear_KB(prolog)

for i in df.index:
    dest = df['Destinations'][i].replace("'", "`").lower()
    prolog.assertz(f"destinations('{dest}')")
    prolog.assertz(f"country('{dest}','{df['country'][i].lower()}')")
    prolog.assertz(f"region('{dest}','{df['region'][i].lower()}')")
    prolog.assertz(f"climate('{dest}','{df['Climate'][i].lower()}')")
    prolog.assertz(f"budget('{dest}','{df['Budget'][i].lower()}')")
    prolog.assertz(f"activity('{dest}','{df['Activity'][i].lower()}')")
    prolog.assertz(f"demographics('{dest}','{df['Demographics'][i].lower()}')")
    prolog.assertz(f"duration('{dest}','{df['Duration'][i].lower()}')")
    prolog.assertz(f"cuisine('{dest}','{df['Cuisine'][i].lower()}')")
    prolog.assertz(f"history('{dest}','{df['History'][i].lower()}')")
    prolog.assertz(f"natural_wonder('{dest}','{df['Natural Wonder'][i].lower()}')")
    prolog.assertz(f"accommodation('{dest}','{df['Accommodation'][i].lower()}')")
    prolog.assertz(f"language('{dest}','{df['Language'][i].lower()}')")



# TODO 2: extract unique features from the Destinations.csv and save them in a dictionary
################################################################################################

key_words = dict() 
for col in ['Destinations', 'country', 'region', 'Climate', 'Budget', 'Activity', 'Demographics', 'Duration', 'Cuisine', 'History', 'Natural Wonder', 'Accommodation', 'Language']:
    key_words[col] = list(map(lambda x: x.lower(), df[col].unique().tolist()))

    

# TODO 5: create the knowledgebase of the city and its connected destinations using Adjacency_matrix.csv

prolog_connection = Prolog()

prolog_connection.retractall("directly_connected(_,_)")
prolog_connection.retractall("connected(_,_)")
prolog_connection.retractall("is_path1(_,_,_)")
prolog_connection.retractall("is_path2(_,_,_)")
prolog_connection.retractall("middle_city(_,_,_)")

        
mx = adj.to_numpy()


for row in mx:
    for i, col in enumerate(row[1:]):
        if col == 1:
            city1 = row[0].replace("'", "`").lower()
            city2 = mx[i][0].replace("'", "`").lower()
            prolog_connection.assertz(f"directly_connected('{city1}', '{city2}')")
            prolog_connection.assertz(f"directly_connected('{city2}', '{city1}')")

# prolog_connection.assertz("directly_connected(X, Y) :- directly_connected(Y, X)")
prolog_connection.assertz("connected(X, Y) :- directly_connected(X, Y)")
prolog_connection.assertz("connected(X, Y) :- directly_connected(X, Z), directly_connected(Z, Y)")
# prolog_connection.assertz("connected(X1, X4) :- directly_connected(X1, X2), directly_connected(X2, X3), directly_connected(X3, X4)")
# prolog_connection.assertz("connected(X1, X5) :- directly_connected(X1, X2), directly_connected(X2, X3), directly_connected(X3, X4), directly_connected(X4, X5)")
# prolog_connection.assertz("connected(X1, X6) :- directly_connected(X1, X2), directly_connected(X2, X3), directly_connected(X3, X4), directly_connected(X4, X5), directly_connected(X5, X6)")




# prolog_connection.assertz("connected(X, Y) :- connected(X, Z), connected(Z, Y), not(X=Y)")
# prolog_connection.assertz("connected(X, X)")

prolog_connection.assertz("is_path1(X, Y, 'true') :- directly_connected(X, Y)")
prolog_connection.assertz("is_path1(X, Y, 'false') :- not(directly_connected(X, Y))")

prolog_connection.assertz("is_path2(X, Y, 'true') :- connected(X, Y)")
prolog_connection.assertz("is_path2(X, Y, 'false') :- not(connected(X, Y))")

prolog_connection.assertz("middle_city(X, Y, Z) :- directly_connected(X, Z), directly_connected(Y, Z)")



# prolog_connection.assertz("connected(X, Y) :- dfs(X, Y, [X])")
# prolog_connection.assertz("dfs(X, X, _)")
# prolog_connection.assertz("dfs(X, Y, Visited) :- connected(X, Z), \+member(Z, Visited), dfs(Z, Y, [Z|Visited])")

# prolog_connection.assertz("directly_connected(tokyo, rome)")
# prolog_connection.assertz("directly_connected(tehran, rome)")
# prolog_connection.assertz("directly_connected(tehran, berlin)")

# path = list(prolog_connection.query("is_path(tokyo, berlin, R)"))
# path = list(prolog_connection.query("is_path(tokyo, tehran, R)"))
# path = list(prolog_connection.query("connected(berlin, X)"))
# print(path)
    

In [5]:


class App(tkinter.Tk):

    APP_NAME = "map_view_demo.py"
    WIDTH = 800
    HEIGHT = 750  # This is now the initial size, not fixed.

    def __init__(self, *args, **kwargs):
        tkinter.Tk.__init__(self, *args, **kwargs)

        self.title(self.APP_NAME)
        self.geometry(f"{self.WIDTH}x{self.HEIGHT}")

        # Configure the grid
        self.grid_columnconfigure(0, weight=1)
        self.grid_rowconfigure(0, weight=1)  # Text area and submit button combined row
        self.grid_rowconfigure(1, weight=4)  # Map row

        # Upper part: Text Area and Submit Button
        self.text_area = tkinter.Text(self, height=5)  # Reduced height for text area
        self.text_area.grid(row=0, column=0, pady=(10, 0), padx=10, sticky="nsew")

        self.submit_button = tkinter.Button(self, text="Submit", command=self.process_text)
        self.submit_button.grid(row=0, column=0, pady=(0, 10), padx=10, sticky="se")  # Placed within the same cell as text area

        # Lower part: Map Widget
        self.map_widget = TkinterMapView(self)
        self.map_widget.grid(row=1, column=0, sticky="nsew")

        self.marker_list = []  # Keeping track of markers
        self.marker_path = None


    def __init__(self, *args, **kwargs):
        tkinter.Tk.__init__(self, *args, **kwargs)

        self.title(self.APP_NAME)
        self.geometry(f"{self.WIDTH}x{self.HEIGHT}")

        # Configure the grid
        self.grid_columnconfigure(0, weight=1)
        self.grid_rowconfigure(0, weight=1)  # Text area can expand/contract.
        self.grid_rowconfigure(1, weight=0)  # Submit button row; doesn't need to expand.
        self.grid_rowconfigure(2, weight=3)  # Map gets the most space.

        # Upper part: Text Area and Submit Button
        self.text_area = tkinter.Text(self)
        self.text_area.grid(row=0, column=0, pady=10, padx=10, sticky="nsew")
        
        self.submit_button = tkinter.Button(self, text="Submit", command=self.process_text)
        self.submit_button.grid(row=1, column=0, pady=10, sticky="ew")

        # Lower part: Map Widget
        self.map_widget = TkinterMapView(self)
        self.map_widget.grid(row=2, column=0, sticky="nsew")

        self.marker_list = []  # Keeping track of markers


    def check_connections(self, results):
        print(f"finding tour for: {results}")
        locations = []
        mark = {}
        for result in results:
            mark[result] = False
            
        for result in results:
            if not mark[result]:
                mark[result] = True 
                tmp = [result]
                f = True
                while f:
                    f = False
                    tmp2 = tmp.copy()
                    for x in tmp:
                        for y in results:
                            if y not in tmp2:
                                query = "is_path1" + f"('{x}', '{y}', Res)"
                                result = list(prolog_connection.query(query))
                                result = result[0]['Res']
                                if result == 'true':
                                    f = True
                                    tmp2.append(y)
                                    mark[y] = True
                                    continue

                                query = "is_path2" + f"('{x}', '{y}', Res)"
                                result = list(prolog_connection.query(query))
                                result = result[0]['Res']
                                if result == 'true':
                                    f = True
                                    tmp2.append(y)
                                    mark[y] = True
                                    query = "middle_city" + f"('{x}', '{y}', Res)"
                                    result = list(prolog_connection.query(query))
                                    city = result[0]['Res']
                                    if city not in tmp2 :
                                        tmp2.append(city)
                                        mark[city] = True

                                "Mamad zahmat mikeshe :)"
                    tmp = tmp2.copy()

                locations.append(tmp.copy())
                                
                

            
        # query = f"connected({results[0]}, X)"
        # self.search(query)
        # return results
        best_locations = []
        best_dest = 0
        best_len = 1e9
        for loc in locations:
            dest = 0
            for res in results:
                if res in loc:
                    dest += 1
            if dest > best_dest:
                best_dest = dest
                best_locations = loc
                best_len = len(loc)
            elif dest == best_dest and len(loc) < best_len:
                best_len = len(loc)
                best_locations = loc

        return best_locations

        # return locations

    def process_text(self):
        """Extract locations from the text area and mark them on the map."""
        text = self.text_area.get("1.0", "end-1c")  # Get text from text area
        results = self.extract_locations(text)  # Extract locations (you may use a more complex method here)

        # TODO 6: if the number of destinations is less than 6 mark and connect them
        ################################################################################################
        if len(results) > 5:
            print('Not enough amount of information provided.\nPlease add some details.')
            return
        if len(results) == 0:
            print('No possible tour for you!')
            return

        # TODO 4: create the query based on the extracted features of user desciption 
        ################################################################################################
        locations = self.check_connections(results)

        print(f"recommended tour: {locations}")

        # locations = ['mexico_city','rome' ,'brasilia']
        self.mark_locations(locations)




    def mark_locations(self, locations):
        """Mark extracted locations on the map."""
        self.map_widget.delete_all_marker()
        self.map_widget.delete_all_path()
        self.map_widget.delete_all_polygon()
        self.marker_list.clear()

        for address in locations:
            marker = self.map_widget.set_address(address, marker=True)
            if marker:
                self.marker_list.append(marker)
        if len(self.marker_list) > 1:
            self.connect_marker()
        self.map_widget.set_zoom(1)  # Adjust as necessary, 1 is usually the most zoomed out


    def connect_marker(self):
        position_list = []

        for marker in self.marker_list:
            position_list.append(marker.position)

        if hasattr(self, 'marker_path') and self.marker_path is not None:
            self.map_widget.delete(self.marker_path)

        if len(position_list) > 0:
            self.marker_path = self.map_widget.set_path(position_list)

    def extract_locations(self, text):
        """Extract locations from text. A placeholder for more complex logic."""
        # Placeholder: Assuming each line in the text contains a single location name
        # TODO 3: extract key features from user's description of destinations
        ################################################################################################
        results = list()
        for key in ['Destinations', 'country', 'region', 'Climate', 'Budget', 'Activity', 'Demographics', 'Duration', 'Cuisine', 'History', 'Natural Wonder', 'Accommodation', 'Language']:
            for word in key_words[key]:
                if word in text:
                    if key == 'Destinations':
                        results.append([word])
                        continue

                    query = key.lower().replace(" ", "_") + f"(City, '{word}')"
                    result = list(prolog.query(query))
                    results.append([])
                    for x in result:
                        results[-1].append(x['City'])
                        
        final_results = []
        if results:
            final_results = results[0]
            for lst in results[1:]:
                final_results = list(set(final_results) & set(lst))

        # return [line.strip() for line in text.split('\n') if line.strip()]
        return final_results

    def start(self):
        self.mainloop()
    
    def search(self, query):
        connected = list(prolog.query(query))
        lst = []
        for dest in connected:
            lst.append(dest["X"])

        return lstlocations[0]





In [6]:
# Don't run for now :)
if __name__ == "__main__":
    app = App()
    app.start()

finding tour for: ['beijing', 'shanghai', 'guangzhou', 'chengdu', 'xi`an']
recommended tour: ['beijing', 'shanghai', 'guangzhou', 'chengdu', 'xi`an', 'tokyo']
